In [2]:
import csv
import tensorflow as tf
import itertools
import operator
import numpy as np
import nltk
import sys
from datetime import datetime

In [ ]:
vocabulary_size = 8000
unknown_token = "UNKNOWN_TOKEN"
sentence_start_token = "SENTENCE_START"
sentence_end_token = "SENTENCE_END"

with open('Chatbot.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    questions = [x[0] for x in reader]
with open('Chatbot.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    answers = [x[1] for x in reader]
with open('Chatbot.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    label = [int(x[2]) for x in reader]

In [ ]:
questions

In [ ]:
answers

In [ ]:
label

In [ ]:
tokenized_sentences = [nltk.word_tokenize(sent) for sent in sentences]
print(tokenized_sentences[0], len(tokenized_sentences))

In [ ]:
with open('Chatbot.csv', 'r',encoding='utf-8') as f:
    reader = csv.reader(f)
    next(reader)
    intents = [int(i[3]) for i in reader]

In [ ]:
intents, len(intents)

In [ ]:
import pickle
with open('data.txt', 'wb') as f:
    pickle.dump(tokenized_sentences, f)

In [ ]:
import pickle
with open ('words.txt', 'rb') as fp:
    wordlist = pickle.load(fp)
with open('embeddings.txt','rb') as fp:
    embeddings = pickle.load(fp)

In [ ]:
len(wordlist), len(embeddings)

In [ ]:
wordlist.append('UNKNOWN_TOKEN')
embeddings = np.vstack([embeddings, np.zeros(150)])

In [ ]:
len(wordlist), len(embeddings)

In [ ]:
word_to_index = dict([(w,i) for i,w in enumerate(wordlist)])

In [ ]:
word_to_index

In [ ]:
wordlist

In [ ]:
for i, sent in enumerate(tokenized_sentences):
    tokenized_sentences[i] = [w if w in word_to_index else unknown_token for w in sent]

In [ ]:
tokenized_sentences

In [ ]:
import numpy as np
X_train = [[word_to_index[w] for w in sent] for sent in tokenized_sentences]

In [ ]:
X_train

In [ ]:
np.median([len(row) for row in X_train])

In [ ]:
import tensorflow as tf
import numpy as np
max_length = max([len(row) for row in X_train])
x_padded = np.array([row + [0] * (max_length - len(row)) for row in X_train])

In [ ]:
x_padded

In [ ]:
np.arange(3)

In [ ]:
intents

In [ ]:
# intents = np.array(intents
labels = np.zeros((133,16))
labels[np.arange(133), intents] = 1

In [ ]:
len(x_padded), len(labels)

In [ ]:
x_train = x_padded[:100]
y_train = labels[:100]
x_test = x_padded[100:]
y_test = labels[100:]
len(x_train),len(y_train),len(x_test),len(y_test)

In [ ]:
len(x_train[0]),len(y_train[0])

In [ ]:
tf.reset_default_graph()
from tensorflow.contrib import rnn 

num_epochs = 30    # no of times to run through the whole datset
batch_size = 30    # 50 sentences at a time are sent for training
num_steps = 11    # the lenght of each sentence is 35
state_size = 150   # the size of each word vector that we have got from the Glove model is 200. 
vocab_size = 184  # out of 3700 odd unique words, we have used only 3500 words as vocab size.
num_classes = 16   # we have 5 classes 

# Placeholders
x = tf.placeholder(tf.int32, [batch_size, None])
y = tf.placeholder(tf.int32, [batch_size, None])

def train_neural_network(x): 
    
        def next_batch(step):   # get the next batch at each iteration 
            p,q = x_train[batch_size*step:batch_size*(step+1)], y_train[batch_size*step:batch_size*(step+1)]
            return p,q
        
        def test_batch(stp):    # gets the test batch
            a,b = x_test[batch_size*stp:batch_size*(stp+1)], y_test[batch_size*stp:batch_size*(stp+1)]
            return a,b
              
        def lstm_neural_network(x):  # the actual LSTM model
            
            # Embedding layer, this where each word is converted in to a word vector. 
            rnn_inputs = tf.nn.embedding_lookup(embeddings, x)
            print("This is the input for LSTM:", rnn_inputs)
            rnn_inputs = tf.transpose(rnn_inputs, [1,0,2])
            rnn_inputs = tf.reshape(rnn_inputs, [-1, state_size])
            rnn_inputs = tf.split(rnn_inputs, num_steps, 0)
            print("This the input after Processing the dimensions for LSTM:", rnn_inputs)
            
            # LSTM Cell used for training the model 
            cell = rnn.BasicLSTMCell(num_units=state_size, state_is_tuple=True) 
#             cell = rnn.LSTMCell(num_units=state_size, state_is_tuple=True)
            cell = rnn.DropoutWrapper(cell=cell, output_keep_prob=0.4)
            cell = rnn.MultiRNNCell(cells=[cell] * 3, state_is_tuple=True)
            rnn_outputs, rnn_states = rnn.static_rnn(cell, rnn_inputs, dtype=tf.float64)
            
            
            
            
            # the weight variable 
            W = tf.get_variable("W", [state_size, num_classes], dtype=tf.float64)
            
            # the bias variable
            b = tf.get_variable("b", [num_classes], dtype=tf.float64)
            
            # outputs from the LSTM model without softmax applied. 
            
            logits = tf.matmul(rnn_outputs[-1], W) + b
            
            # when softmax is applied for the logits, we get the probabilities for each class. can be used directly in testing. 
            probs = tf.nn.softmax(logits)
            return logits, probs

        prediction, probability  = lstm_neural_network(x)
        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y))  # cal loss
        optimizer = tf.train.AdamOptimizer(0.01).minimize(cost)   # learning rate 0.01 by default. 
        saver = tf.train.Saver()
        
        with tf.Session() as sess:
            sess.run(tf.initialize_all_variables())
            for epoch in range(num_epochs):
                epoch_loss = 0 
                for step in range(int(len(x_train)/batch_size)):
                    epoch_x, epoch_y = next_batch(step)
                    _, c = sess.run([optimizer, cost], feed_dict={x: epoch_x, y: epoch_y})
                    epoch_loss += c
                print('Epoch', epoch, 'completed out of',num_epochs,'loss:',epoch_loss)
            save_path = saver.save(sess, "./model")
            print("Model saved in file: %s" % save_path)
            
            for stp in range(int(len(x_test)/batch_size)):
                s,u = test_batch(stp)
                predictedclass = np.argmax(probability.eval({x: s, y: u}),1)
                actualclass = np.argmax(y.eval({x: s, y: u}),1)
                print("Predicted", predictedclass)
                print("Actual", actualclass)
#                 lbl = ['Ans1','Ans2','Ans3','Ans4','Ans5','Ans6','Ans7','Ans8','Ans9','Ans10',
#                        'Ans11','Ans12','Ans13','Ans14','Ans15','Ans16']
#                 for i,j in enumerate(X_train[100:]):
#                     print("Test Sentence:\n%s" % (' '.join([wordlist[x] for x in j])))
#                     print("Predicted Class:", lbl[predictedclass[i]],  "\nActual Class:", lbl[actualclass[i]])

In [ ]:
train_neural_network(x)

In [ ]:
sentence = np.array(sentences)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
import numpy as np
from nltk.corpus import stopwords
import string
from sklearn.externals import joblib
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

y = intents[:100]
le = LabelEncoder()
y = le.fit_transform(y)
X = sentence[:100]
tuned_parameters = [{'C': [1, 2, 5, 10, 20, 100], 'kernel': [str('linear')]}]
cv_splits = max(2, min(5, np.min(np.bincount(y))))
classifier = Pipeline([
        ('vectorizer', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        ('clf2', GridSearchCV(SVC(C=1, probability=True),
                        param_grid=tuned_parameters, n_jobs=1,
                        cv=cv_splits, scoring='f1_weighted'))
        ])

clf = classifier.fit(X, y)

In [ ]:
intent_ml = le.inverse_transform(clf.predict(sentences[100:130]))

In [ ]:
intent_ml

### Dual Encoder

In [141]:
vocabulary_size = 8000
unknown_token = "UNKNOWN_TOKEN"
sentence_start_token = "SENTENCE_START"
sentence_end_token = "SENTENCE_END"
import numpy as np

with open('Chatbot.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    questions = [x[0] for x in reader]
    tokenized_questions = [nltk.word_tokenize(sent) for sent in questions]
with open('Chatbot.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    answers = [x[1] for x in reader]
    tokenized_answers = [nltk.word_tokenize(sent) for sent in answers]
with open('Chatbot.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    labels = [int(i[2]) for i in reader]

In [142]:
questions

['What this demo about?',
 'What this demo about?',
 'what all EA platforms this solution supports',
 'how this will be deployed',
 'how this will be deployed',
 'how this will be deployed',
 'Can I get more info about this tool',
 'Can I get more info about this tool',
 'Which all platforms this solution supports',
 'Does this access backend application',
 'Which all platforms this solution supports',
 'what all EA platforms this solution supports',
 'Does this access backend application',
 'where this will be deployed',
 'what if I can not text',
 'how this will be deployed',
 'What this video about?',
 'What this about?',
 'How client can get this solution',
 'How customer can get this solution',
 'Does this connect to Enterprise application',
 'Which all system this solution connects to',
 'What this demo about?',
 'what all EA platforms this solution supports',
 'what all EA platforms this solution supports',
 'what all EA platforms this solution supports',
 'how can we configure 

In [143]:
tokenized_questions

[['What', 'this', 'demo', 'about', '?'],
 ['What', 'this', 'demo', 'about', '?'],
 ['what', 'all', 'EA', 'platforms', 'this', 'solution', 'supports'],
 ['how', 'this', 'will', 'be', 'deployed'],
 ['how', 'this', 'will', 'be', 'deployed'],
 ['how', 'this', 'will', 'be', 'deployed'],
 ['Can', 'I', 'get', 'more', 'info', 'about', 'this', 'tool'],
 ['Can', 'I', 'get', 'more', 'info', 'about', 'this', 'tool'],
 ['Which', 'all', 'platforms', 'this', 'solution', 'supports'],
 ['Does', 'this', 'access', 'backend', 'application'],
 ['Which', 'all', 'platforms', 'this', 'solution', 'supports'],
 ['what', 'all', 'EA', 'platforms', 'this', 'solution', 'supports'],
 ['Does', 'this', 'access', 'backend', 'application'],
 ['where', 'this', 'will', 'be', 'deployed'],
 ['what', 'if', 'I', 'can', 'not', 'text'],
 ['how', 'this', 'will', 'be', 'deployed'],
 ['What', 'this', 'video', 'about', '?'],
 ['What', 'this', 'about', '?'],
 ['How', 'client', 'can', 'get', 'this', 'solution'],
 ['How', 'customer', 

In [144]:
tokenized_answers

[['This',
  'is',
  'a',
  'Chatbot',
  'based',
  'Self-Heal',
  'Solution',
  'for',
  'the',
  'Oracle',
  'EBS',
  'Platform'],
 ['Current',
  'Demo',
  'is',
  'for',
  'Oracle',
  'EBS',
  '.',
  'However',
  'Self-Heal',
  'Solution',
  'is',
  'available',
  'for',
  'all',
  'EA',
  'Platforms',
  'like',
  'SAP',
  ',',
  'Salesforce',
  'etc',
  '.'],
 ['This',
  'is',
  'a',
  'Chatbot',
  'based',
  'Self-Heal',
  'Solution',
  'for',
  'the',
  'Oracle',
  'EBS',
  'Platform'],
 ['Pls',
  'reach',
  'out',
  'to',
  '``',
  'Radheshyam',
  'Patil',
  "''",
  'for',
  'such',
  'details',
  '/',
  'artifacts'],
 ['This',
  'is',
  'a',
  'Python',
  'based',
  'solution',
  'and',
  'uses',
  'relevant',
  'Open',
  'Source',
  'libraries'],
 ['This',
  'solution',
  'eliminates',
  '100',
  '%',
  'manual',
  'efforts',
  'for',
  'covered',
  'use',
  'cases',
  '.',
  'Hence',
  'total',
  'benefits',
  '=',
  'Summation',
  'of',
  '(',
  'Current',
  'manual',
  'Effo

In [145]:
labels

[1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,


In [146]:
completedata = tokenized_questions.copy()

In [147]:
for i in tokenized_answers:
    completedata.append(i)

In [148]:
len(completedata)

514

In [ ]:
import pickle
with open('questions.txt', 'wb') as f:
    pickle.dump(completedata, f)

In [ ]:
with open ('words.txt', 'rb') as fp:
    wordlist = pickle.load(fp)

In [ ]:
with open('embeddings.txt','rb') as fp:
    embeddings = pickle.load(fp)

In [104]:
word_freq = nltk.FreqDist(itertools.chain(*completedata))

In [105]:
len(word_freq)

305

In [106]:
vocab = word_freq.most_common(305)

In [107]:
vocab

[('.', 254),
 ('this', 196),
 ('to', 175),
 ('is', 168),
 ('for', 142),
 ('Solution', 141),
 ('solution', 104),
 ('This', 100),
 ('can', 97),
 (',', 92),
 ("''", 88),
 ('``', 87),
 ('as', 86),
 ('the', 85),
 ('of', 74),
 ('Self-Heal', 74),
 ('reach', 70),
 ('out', 70),
 ('based', 67),
 ('on', 64),
 ('use', 63),
 (')', 62),
 ('(', 62),
 ('like', 61),
 ('etc', 61),
 ('EA', 60),
 ('a', 59),
 ('be', 59),
 ('and', 59),
 ('all', 55),
 ('/', 51),
 ('Chatbot', 51),
 ('get', 50),
 ('tool', 46),
 ('supports', 45),
 ('Current', 44),
 ('I', 43),
 ('ITSM', 42),
 ('Can', 42),
 ('Ticketing', 41),
 ('deployed', 40),
 ('cases', 35),
 ('your', 35),
 ('JIRA', 35),
 ('Feel', 35),
 ('Oracle', 35),
 ('EBS', 34),
 ('secured', 33),
 ('Platform', 32),
 ('task', 32),
 ('However', 31),
 ('pls', 31),
 ('Demo', 31),
 ('required', 31),
 ('with', 31),
 ('For', 31),
 ('Is', 30),
 ('what', 29),
 ('how', 28),
 ('?', 28),
 ('Hence', 28),
 ('details', 28),
 ('Remedy', 28),
 ('it', 28),
 ('about', 28),
 ('HPSM', 28),
 ('c

In [108]:
index_to_word = [x[0] for x in vocab]
index_to_word.append(unknown_token)

In [109]:
len(index_to_word)

306

In [110]:
word_to_index = dict([(w,i) for i,w in enumerate(index_to_word)])

In [111]:
index_to_word

['.',
 'this',
 'to',
 'is',
 'for',
 'Solution',
 'solution',
 'This',
 'can',
 ',',
 "''",
 '``',
 'as',
 'the',
 'of',
 'Self-Heal',
 'reach',
 'out',
 'based',
 'on',
 'use',
 ')',
 '(',
 'like',
 'etc',
 'EA',
 'a',
 'be',
 'and',
 'all',
 '/',
 'Chatbot',
 'get',
 'tool',
 'supports',
 'Current',
 'I',
 'ITSM',
 'Can',
 'Ticketing',
 'deployed',
 'cases',
 'your',
 'JIRA',
 'Feel',
 'Oracle',
 'EBS',
 'secured',
 'Platform',
 'task',
 'However',
 'pls',
 'Demo',
 'required',
 'with',
 'For',
 'Is',
 'what',
 'how',
 '?',
 'Hence',
 'details',
 'Remedy',
 'it',
 'about',
 'HPSM',
 'connect',
 'requirement',
 'well',
 'On-Campus',
 'tools',
 'On-Cloud',
 'Nitesh',
 'pre-requisites',
 'clients',
 'Awasthi',
 'How',
 'platforms',
 'Does',
 'we',
 'manual',
 'SNow',
 'most',
 'standard',
 'industry',
 'depends',
 'No',
 'connects',
 'What',
 'takes',
 'free',
 'Patil',
 'Radheshyam',
 'artifacts',
 'Which',
 'Pls',
 'such',
 'only',
 'parameters',
 'licensed',
 'Parikh',
 'Cases',
 'f

In [112]:
len(word_to_index)

306

In [113]:
index_to_word[305]

'UNKNOWN_TOKEN'

In [114]:
word_to_index

{'%': 164,
 '&': 150,
 "''": 10,
 "'Structured": 266,
 "'s": 188,
 '(': 22,
 ')': 21,
 '+': 131,
 ',': 9,
 '.': 0,
 '/': 30,
 '100': 178,
 '=': 173,
 '?': 59,
 'AES-256': 120,
 'Application': 267,
 'Are': 222,
 'Automation': 137,
 'Awasthi': 75,
 'Backend': 132,
 'Business': 292,
 'Can': 38,
 'Cases': 101,
 'Chatbot': 31,
 'Cognizant': 146,
 'Conversation': 122,
 'Current': 35,
 'Demo': 52,
 'Does': 78,
 'EA': 25,
 'EAS': 189,
 'EBS': 46,
 'Efforts': 176,
 'End-to-End': 136,
 'Enterprise': 241,
 'Feel': 44,
 'For': 55,
 'HP': 232,
 'HPSM': 65,
 'Hence': 60,
 'How': 76,
 'However': 50,
 'I': 36,
 'IA': 185,
 'ITSM': 37,
 'Implementation': 220,
 'Is': 56,
 'JIRA': 43,
 'Jose': 226,
 'Kunal': 105,
 'Language': 202,
 'Leema': 225,
 'Manager': 240,
 'May': 278,
 'Mundane': 158,
 'NLP': 209,
 'Natural': 207,
 'Need': 237,
 'Nitesh': 72,
 'No': 86,
 'On-Campus': 69,
 'On-Cloud': 71,
 'Open': 152,
 'Oracle': 45,
 'Parikh': 100,
 'Patil': 91,
 'Platform': 48,
 'Platforms': 125,
 'Please': 107,


In [115]:
digits = np.arange(0,306)
ems = np.zeros((306,306))
ems[np.arange(306),digits] = 1

In [116]:
embedded = ems

In [149]:
wordlist.append('UNKNOWN_TOKEN')

In [150]:
embeddings = np.vstack([embeddings, np.zeros(200)])

In [151]:
len(wordlist)

306

In [152]:
word_to_index = dict([(w,i) for i,w in enumerate(wordlist)])

In [153]:
word_to_index

{'%': 63,
 '&': 101,
 "''": 108,
 "'Structured": 52,
 "'s": 255,
 '(': 57,
 ')': 205,
 '+': 186,
 ',': 141,
 '.': 76,
 '/': 82,
 '100': 294,
 '=': 241,
 '?': 20,
 'AES-256': 249,
 'Application': 14,
 'Are': 169,
 'Automation': 178,
 'Awasthi': 269,
 'Backend': 59,
 'Business': 203,
 'Can': 170,
 'Cases': 252,
 'Chatbot': 259,
 'Cognizant': 278,
 'Conversation': 86,
 'Current': 211,
 'Demo': 96,
 'Does': 51,
 'EA': 197,
 'EAS': 302,
 'EBS': 303,
 'Efforts': 29,
 'End-to-End': 97,
 'Enterprise': 297,
 'Feel': 152,
 'For': 262,
 'HP': 80,
 'HPSM': 194,
 'Hence': 5,
 'How': 18,
 'However': 293,
 'I': 93,
 'IA': 184,
 'ITSM': 104,
 'Implementation': 0,
 'Is': 158,
 'JIRA': 132,
 'Jose': 154,
 'Kunal': 292,
 'Language': 258,
 'Leema': 24,
 'Manager': 275,
 'May': 79,
 'Mundane': 206,
 'NLP': 263,
 'Natural': 251,
 'Need': 198,
 'Nitesh': 109,
 'No': 287,
 'On-Campus': 75,
 'On-Cloud': 193,
 'Open': 225,
 'Oracle': 192,
 'Parikh': 110,
 'Patil': 65,
 'Platform': 232,
 'Platforms': 200,
 'Plea

In [154]:
for i, sent in enumerate(tokenized_questions):
    questions[i] = [w if w in word_to_index else unknown_token for w in sent]

In [155]:
for i, sent in enumerate(tokenized_answers):
    answers[i] = [w if w in word_to_index else unknown_token for w in sent]

In [156]:
import numpy as np
X_question = [[word_to_index[w] for w in sent] for sent in questions]
y_answer = [[word_to_index[w] for w in sent] for sent in answers]

In [157]:
X_question

[[13, 117, 69, 190, 20],
 [13, 117, 69, 190, 20],
 [290, 176, 197, 34, 117, 219, 130],
 [280, 117, 22, 162, 229],
 [280, 117, 22, 162, 229],
 [280, 117, 22, 162, 229],
 [170, 93, 56, 138, 120, 190, 117, 62],
 [170, 93, 56, 138, 120, 190, 117, 62],
 [92, 176, 34, 117, 219, 130],
 [51, 117, 283, 164, 143],
 [92, 176, 34, 117, 219, 130],
 [290, 176, 197, 34, 117, 219, 130],
 [51, 117, 283, 164, 143],
 [179, 117, 22, 162, 229],
 [290, 70, 93, 214, 234, 17],
 [280, 117, 22, 162, 229],
 [13, 117, 33, 190, 20],
 [13, 117, 190, 20],
 [18, 301, 214, 56, 117, 219],
 [18, 277, 214, 56, 117, 219],
 [51, 117, 209, 272, 297, 143],
 [92, 176, 121, 117, 219, 161, 272],
 [13, 117, 69, 190, 20],
 [290, 176, 197, 34, 117, 219, 130],
 [290, 176, 197, 34, 117, 219, 130],
 [290, 176, 197, 34, 117, 219, 130],
 [280, 214, 125, 247, 117, 291, 301],
 [280, 214, 125, 247, 117, 291, 301],
 [280, 214, 125, 247, 117, 291, 301],
 [158, 77, 173],
 [51, 117, 66, 221, 132],
 [51, 117, 100, 132],
 [10, 254, 41, 136, 12]

In [158]:
np.max([len(row) for row in X_question]),np.max([len(row) for row in y_answer])

(11, 28)

In [159]:
y_answer

[[213, 64, 9, 259, 139, 150, 36, 291, 222, 192, 303, 232],
 [211,
  96,
  64,
  291,
  192,
  303,
  76,
  293,
  150,
  36,
  64,
  167,
  291,
  176,
  197,
  200,
  30,
  28,
  141,
  111,
  207,
  76],
 [213, 64, 9, 259, 139, 150, 36, 291, 222, 192, 303, 232],
 [175, 126, 185, 272, 131, 165, 65, 108, 291, 237, 44, 82, 147],
 [213, 64, 9, 188, 139, 219, 257, 58, 223, 225, 83, 95],
 [213,
  219,
  43,
  294,
  63,
  274,
  253,
  291,
  289,
  49,
  40,
  76,
  5,
  88,
  74,
  241,
  128,
  60,
  57,
  211,
  274,
  29,
  140,
  106,
  295,
  49,
  21,
  205],
 [36,
  214,
  162,
  229,
  75,
  119,
  153,
  119,
  193,
  139,
  106,
  261,
  25,
  76,
  262,
  118,
  202,
  72,
  126,
  185,
  272,
  131,
  109,
  269,
  108],
 [213, 64, 9, 188, 139, 219, 257, 58, 223, 225, 83, 95],
 [213, 64, 9, 259, 139, 150, 36, 291, 222, 192, 303, 232],
 [213, 64, 9, 259, 139, 150, 36, 291, 222, 192, 303, 232],
 [211,
  96,
  64,
  291,
  192,
  303,
  76,
  293,
  150,
  36,
  64,
  167,
  291

In [160]:
max_length =  28
x_padded = []
y_padded = []

for row in X_question:
    if len(row) <= max_length:
        x_padded.append(list(row) + [305] * (max_length - len(row)))
    else :
        x_padded.append(row[:28])
        
for row in y_answer:
    if len(row) <= max_length:
        y_padded.append(list(row) + [305] * (max_length - len(row))) 
    else: 
        y_padded.append(row[:28])

In [161]:
len(x_padded)

257

In [162]:
len(y_padded)

257

In [166]:
tf.reset_default_graph()

from tensorflow.contrib import rnn 
hm_epochs = 25
state_size = 200
vocab_size = 306
num_steps = 28
batch_size = 25
# Placeholders
context = tf.placeholder(tf.int32, [None, num_steps], name = "context") # [batch_size, num_steps]
utterance = tf.placeholder(tf.int32, [None,num_steps], name = "utterance")
targets = tf.placeholder(tf.int32, [None], name = "targets")
target = tf.placeholder(tf.int32, [None], name = "target")

def testbatch_pp():
    with open('Chatbot_test.csv', 'r') as f:
        reader = csv.reader(f)
        next(reader)
        questions = [x[0] for x in reader]
        testquestions = [nltk.word_tokenize(sent) for sent in questions]
    with open('Chatbot_test.csv', 'r') as f:
        reader = csv.reader(f)
        next(reader)
        answers = [x[1] for x in reader]
        testanswers = [nltk.word_tokenize(sent) for sent in answers]
    with open('Chatbot_test.csv', 'r') as f:
        reader = csv.reader(f)
        next(reader)
        testlabels = [int(i[2]) for i in reader]

    for i, sent in enumerate(testquestions):
        testquestions[i] = [w if w in word_to_index else unknown_token for w in sent]
    
    for i, sent in enumerate(testanswers):
        testanswers[i] = [w if w in word_to_index else unknown_token for w in sent]
    
    X_testquestion = np.asarray([[word_to_index[w] for w in sent] for sent in testquestions])
    y_testanswer = np.asarray([[word_to_index[w] for w in sent] for sent in testanswers])
    
    x = X_testquestion
    y = y_testanswer
    max_length =  28
    x_testpadded = []
    y_testpadded = []

    for row in x:
        if len(row) <= max_length:
            x_testpadded.append(list(row) + [305] * (max_length - len(row)))
        else :
            x_testpadded.append(row[:28])

    for row in y:
        if len(row) <= max_length:
            y_testpadded.append(list(row) + [305] * (max_length - len(row))) 
        else: 
            y_testpadded.append(row[:28])
    

    print(len(x_testpadded), len(y_testpadded))
    
    return x_testpadded,y_testpadded,testlabels 


def next_batch(step):
    p,q = x_padded[batch_size*step:batch_size*(step+1)], y_padded[batch_size*step:batch_size*(step+1)]
    t = labels[batch_size*step:batch_size*(step+1)]
    return p,q,t
        
def dual_encoder_model(context,utterance,targets, batch_size):
    
    #Embedding the question and answer
    context_embedded = tf.nn.embedding_lookup(embeddings, context, name="embed_context")
    utterance_embedded = tf.nn.embedding_lookup(embeddings, utterance, name="embed_utterance")
    print(" This is the embeddings for both Q&A:", context_embedded, utterance_embedded)

    with tf.variable_scope("rnn") as vs:
        cell = tf.contrib.rnn.BasicLSTMCell(state_size)
        cell = tf.contrib.rnn.DropoutWrapper(cell=cell, output_keep_prob = 0.5) 
#         cell = tf.contrib.rnn.BasicLSTMCell(num_units=state_size, state_is_tuple= True )
#         cell = tf.contrib.rnn.DropoutWrapper(cell=cell, output_keep_prob = 0.5) 
#         cell = tf.contrib.rnn.MultiRNNCell(cells= [cell]*3, state_is_tuple= True)

    #feed inputs to dynamic rnn for training    
    rnn_inputs = tf.concat([context_embedded, utterance_embedded],0)
    rnn_inputs = tf.transpose(rnn_inputs, [1,0,2])
    rnn_inputs = tf.reshape(rnn_inputs, [-1, state_size])
    rnn_inputs = tf.split(rnn_inputs, num_steps, 0)

    rnn_outputs, rnn_states = rnn.static_rnn(cell, rnn_inputs, dtype=tf.float64)

    encoding_context, encoding_utterance = tf.split(rnn_states.h,2,0)

    print("\nThis is the encoding context:", encoding_context)
    print("\nThis is the encoding utterance:", encoding_utterance)

    with tf.variable_scope("prediction") as vs:
        M = tf.get_variable("M",
            shape=[state_size, state_size],
            initializer=tf.truncated_normal_initializer())

    # "Predict" a  response: c * M
    generated_response = tf.matmul(encoding_context, tf.cast(M, dtype = tf.float64))
    print("\nThis is the generated response:", generated_response)
    generated_response = tf.expand_dims(generated_response, 2)
    print("\nThis is the generated response:", generated_response)
    encoding_utterance = tf.expand_dims(encoding_utterance, 2)
    print("\nThis is the encoding utterance:", encoding_utterance)
    # Dot product between generated response and actual response
    # (c * M) * r
    logit = tf.matmul(generated_response, encoding_utterance, True)
    print("\nThis is the logits before squeeze:", logit)
    logits = tf.squeeze(logit, [2])
    print("\nThis is the logits after squeeze:", logits)

    # Apply sigmoid to convert logits to probabilities
    probs = tf.sigmoid(logits, name = "Sigmoid")   
    print("\nThis is the probability after sigmoid:", probs)
    return tf.reshape(logits, [-1]), tf.reshape(probs, [-1])
        
def train_encoder(training):
    global context,utterance,targets, batch_size
    if training == True: 
        batch_size = 20
        prediction, probs = dual_encoder_model(context,utterance,targets,batch_size)
        # Calculate the binary cross-entropy loss
        print("\nThis is the prediction size:", prediction)
        print("\nThis is the target size:", targets)
        losses = tf.nn.sigmoid_cross_entropy_with_logits(logits = prediction, labels = tf.cast(targets, dtype = tf.float64))

        # Mean loss across the batch of examples
        cost = tf.reduce_mean(losses, name="cost")
        optimizer = tf.train.AdamOptimizer().minimize(cost)      

        saver = tf.train.Saver()
        with tf.Session() as sess:
            sess.run(tf.initialize_all_variables())
            for epoch in range(hm_epochs):
                epoch_loss = 0 
                for step in range(int(len(x_padded)/batch_size)):
                    ques, ans,labl = next_batch(step)
                    _, c, logts, prob = sess.run([optimizer, cost, prediction, probs], feed_dict={context: ques,
                                                                  utterance: ans,  
                                                                  targets: labl})
                    epoch_loss += c

                print('Epoch:', epoch, 'completed out of:',hm_epochs,'loss:',epoch_loss)
            save_path = saver.save(sess, "./Interviewmodel")
            print("Model saved in file: %s" % save_path)  
            #qe, an,lbl = test_batch(0)
            #probas = sess.run(probs, feed_dict={context: qe, utterance: an, target: lbl})
            #print("Probs:", probas)
    else:
        testx, testy, labled = testbatch_pp()
        batch_sise = len(testx)
        print("This is test dataset:", testx[1])
        for i in testy:
            print("x:\n%s\n" % (" ".join([index_to_word[x] for x in i])))
        sess=tf.Session()    
        #First let's load meta graph and restore weights
        saver = tf.train.import_meta_graph('Interviewmodel.meta')
        saver.restore(sess,tf.train.latest_checkpoint('./'))
        sess.run(tf.global_variables_initializer())
        graph = tf.get_default_graph()

        context = graph.get_tensor_by_name("context:0") 
        utterance = graph.get_tensor_by_name("utterance:0")
        target = graph.get_tensor_by_name("target:0")

        op_to_restore = graph.get_tensor_by_name("Sigmoid:0")
        #qe,an,lbl = test_batch(0)
        feed_dict={context: testx, utterance: testy, target: labled}
        result = sess.run(op_to_restore,feed_dict)
        print(result)
        print(np.argmax(list(itertools.chain.from_iterable(result))))

In [167]:
train_encoder(training=True)

 This is the embeddings for both Q&A: Tensor("embed_context:0", shape=(?, 28, 200), dtype=float64) Tensor("embed_utterance:0", shape=(?, 28, 200), dtype=float64)

This is the encoding context: Tensor("split_1:0", shape=(?, 200), dtype=float64)

This is the encoding utterance: Tensor("split_1:1", shape=(?, 200), dtype=float64)

This is the generated response: Tensor("MatMul:0", shape=(?, 200), dtype=float64)

This is the generated response: Tensor("ExpandDims:0", shape=(?, 200, 1), dtype=float64)

This is the encoding utterance: Tensor("ExpandDims_1:0", shape=(?, 200, 1), dtype=float64)

This is the logits before squeeze: Tensor("MatMul_1:0", shape=(?, 1, 1), dtype=float64)

This is the logits after squeeze: Tensor("Squeeze:0", shape=(?, 1), dtype=float64)

This is the probability after sigmoid: Tensor("Sigmoid:0", shape=(?, 1), dtype=float64)

This is the prediction size: Tensor("Reshape_1:0", shape=(?,), dtype=float64)

This is the target size: Tensor("targets:0", shape=(?,), dtype=in

In [169]:
train_encoder(training=False)

15 15
This is test dataset: [290, 176, 197, 34, 117, 219, 130, 305, 305, 305, 305, 305, 305, 305, 305, 305, 305, 305, 305, 305, 305, 305, 305, 305, 305, 305, 305, 305]
x:
demo about , messaging Yes & I model Are which deployment HP UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN

x:
here such about model which deployment How expect & I about Third model Efforts time Processing / and tasks connected Natural How UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN

x:
or How & about only 100 I How Solution Current total in-house system time Implementation Technology apart Are bot How UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN

x:
demo I about takes Automation Technology ? time HP Team Use most more EAS UNKNOWN_TOKEN 

In [41]:
a = tf.placeholder(tf.float32, shape=[None], name="input_placeholder_a")
b = tf.placeholder(tf.float32, shape=[None], name="input_placeholder_b")
normalize_a = tf.nn.l2_normalize(a,0)        
normalize_b = tf.nn.l2_normalize(b,0)
cos_similarity=tf.reduce_sum(tf.multiply(normalize_a,normalize_b))
sess=tf.Session()
cos_sim=sess.run(cos_similarity,feed_dict={a:[1,2,3],b:[2,4,6]})

In [42]:
cos_sim

0.99999988

In [135]:
from gensim.models import Word2Vec

C:\Anaconda3\lib\site-packages\gensim-2.3.0-py3.5-win-amd64.egg\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [136]:
min_count = 1
size = 200
window = 4
model = Word2Vec(completedata, min_count=min_count, size=size, window=window)

C:\Anaconda3\lib\site-packages\gensim-2.3.0-py3.5-win-amd64.egg\gensim\models\word2vec.py:813: UserWarning: C extension not loaded for Word2Vec, training will be slow. Install a C compiler and reinstall gensim for fast training.
  warnings.warn("C extension not loaded for Word2Vec, training will be slow. "


In [137]:
wordlist = list(model.wv.vocab.keys())

In [138]:
wordlist

['Implementation',
 'parameters',
 'automated',
 'there',
 'etc..',
 'Hence',
 'in-house',
 'free',
 'any',
 'a',
 'which',
 'measures',
 'here',
 'What',
 'Application',
 'or',
 'depends',
 'text',
 'How',
 'programming',
 '?',
 'case',
 'will',
 'words',
 'Leema',
 'requirement',
 'these',
 'SNow',
 'SAP',
 'Efforts',
 'like',
 'that',
 'language',
 'video',
 'platforms',
 'internally',
 'Solution',
 'instead',
 'implementation',
 'Video',
 'cases',
 'being',
 'Technology',
 'eliminates',
 'details',
 'coded',
 'know',
 'licensed',
 'point',
 'use',
 'do',
 'Does',
 "'Structured",
 'price',
 'an',
 'texting',
 'get',
 '(',
 'uses',
 'Backend',
 'of',
 'implemented',
 'tool',
 '%',
 'is',
 'Patil',
 'integrate',
 'type',
 'Processing',
 'demo',
 'if',
 'shows',
 'pls',
 'in-campus',
 'benefits',
 'On-Campus',
 '.',
 'Remedy',
 'benefit',
 'May',
 'HP',
 'similar',
 '/',
 'Source',
 'slider',
 'voice',
 'Conversation',
 'channel',
 'total',
 'mainly',
 'hard',
 'chatbot',
 'Which',
 'I

In [139]:
embeddings = [model[i] for i in wordlist]

In [140]:
len(embeddings)

305